# Linear Regression - Crew Members on Ship

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 


In [51]:
from pyspark.sql import SparkSession

In [52]:
spark = SparkSession.builder.appName('cruise').getOrCreate()

In [53]:
df = spark.read.csv('../data/cruise_ship_info.csv',inferSchema=True,header=True)

In [54]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [60]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [58]:
df.describe().show(n=5, truncate=False, vertical=True)

-RECORD 0-------------------------------
 summary           | count              
 Ship_name         | 158                
 Cruise_line       | 158                
 Age               | 158                
 Tonnage           | 158                
 passengers        | 158                
 length            | 158                
 cabins            | 158                
 passenger_density | 158                
 crew              | 158                
-RECORD 1-------------------------------
 summary           | mean               
 Ship_name         | Infinity           
 Cruise_line       | null               
 Age               | 15.689873417721518 
 Tonnage           | 71.28467088607599  
 passengers        | 18.45740506329114  
 length            | 8.130632911392404  
 cabins            | 8.830000000000005  
 passenger_density | 39.90094936708861  
 crew              | 7.794177215189873  
-RECORD 2-------------------------------
 summary           | stddev             
 Ship_name      

## Dealing with the Cruise_line categorical variable
Ship Name is a useless arbitrary string, but the cruise_line itself may be useful. Let's make it into a categorical variable!

In [59]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [63]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
indexed = indexer.fit(df).transform(df)
indexed.show(25)

+-----------+-----------+---+------------------+----------+------+------+-----------------+-----+----------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density| crew|cruise_cat|
+-----------+-----------+---+------------------+----------+------+------+-----------------+-----+----------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64| 3.55|      16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64| 3.55|      16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8|  6.7|       1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99| 19.1|       1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36| 10.0|       1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29|  9.2|       1.0|
|    Elation|   Car

In [64]:
from pyspark.ml.feature import VectorAssembler

In [65]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [66]:
assembler = VectorAssembler(
  inputCols=['Age',
             'Tonnage',
             'passengers',
             'length',
             'cabins',
             'passenger_density',
             'cruise_cat'],
    outputCol="features")

In [69]:
output = assembler.transform(indexed)
output.show(n=5, truncate=False, vertical=True)

-RECORD 0---------------------------------------------------------------
 Ship_name         | Journey                                            
 Cruise_line       | Azamara                                            
 Age               | 6                                                  
 Tonnage           | 30.276999999999997                                 
 passengers        | 6.94                                               
 length            | 5.94                                               
 cabins            | 3.55                                               
 passenger_density | 42.64                                              
 crew              | 3.55                                               
 cruise_cat        | 16.0                                               
 features          | [6.0,30.276999999999997,6.94,5.94,3.55,42.64,16.0] 
-RECORD 1---------------------------------------------------------------
 Ship_name         | Quest                         

In [70]:
output.select("features", "crew").show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [72]:
final_data = output.select("features", "crew")

In [73]:
train_data,test_data = final_data.randomSplit([0.8,0.2])

In [74]:
from pyspark.ml.regression import LinearRegression
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='crew')

In [75]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [76]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.001826873723669741,0.0069754093180174215,-0.13555693363834198,0.394045519726484,0.869082293066353,0.0033330780844575128,0.04652644522602036] Intercept: -1.4147972761208418


In [77]:
test_results = lrModel.evaluate(test_data)

In [78]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

RMSE: 1.0371148434343282
MSE: 1.075607198471811
R2: 0.9187972527417032


In [81]:
predictions = lrModel.transform(test_data)

In [82]:
predictions.show()

+--------------------+-----+------------------+
|            features| crew|        prediction|
+--------------------+-----+------------------+
|[4.0,220.0,54.0,1...| 21.0|21.051040419597214|
|[5.0,122.0,28.5,1...|  6.7| 5.999164436242432|
|[6.0,110.23899999...| 11.5| 11.14404251658495|
|[7.0,89.6,25.5,9....| 9.87|11.051095636567602|
|[9.0,59.058,17.0,...|  7.4|  7.51148976783106|
|[9.0,85.0,19.68,9...| 8.69| 9.246172775886462|
|[9.0,90.09,25.01,...| 8.69| 9.225432234048222|
|[9.0,110.0,29.74,...| 11.6|12.148976728206653|
|[9.0,113.0,26.74,...|12.38|11.333101629010054|
|[10.0,77.0,20.16,...|  9.0| 8.717296542090068|
|[10.0,81.76899999...| 8.42| 8.728499060556256|
|[11.0,58.6,15.66,...|  7.6| 7.349362398739226|
|[11.0,90.0,22.4,9...| 11.0|10.012708444082808|
|[11.0,138.0,31.14...|11.85|13.005068310263635|
|[12.0,88.5,21.24,...|10.29| 9.458957872113373|
|[12.0,88.5,21.24,...|  9.3|10.380185102763708|
|[12.0,91.0,20.32,...| 9.99| 9.148031126339239|
|[12.0,138.0,31.14...|11.85|13.003241436